In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
import time

In [2]:
train_df = pd.read_pickle('Train_final_data.pkl')
test_df = pd.read_pickle('Test_final_data')

In [14]:
def creating_dfg_variables(train_df,val,test_df):
    # 5. Creating default to gross variables
    
    
    df_supplier_id_ftg      = pd.DataFrame(train_df.groupby('supplier_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'supplier_id_ftg'})
    df_branch_id_ftg        = pd.DataFrame(train_df.groupby('branch_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'branch_id_ftg'})
    df_manufacturer_id_ftg  = pd.DataFrame(train_df.groupby('manufacturer_id')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'manufacturer_id_ftg'})
    df_pincode_id_ftg       = pd.DataFrame(train_df.groupby('Current_pincode_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20  else 0)).rename(columns = {'loan_default': 'pincode_id_ftg'})
    df_state_id_ftg         = pd.DataFrame(train_df.groupby('State_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20  else 0)).rename(columns = {'loan_default': 'state_id_ftg'})
    df_employee_code_id_ftg = pd.DataFrame(train_df.groupby('Employee_code_ID')['loan_default'].apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0)).rename(columns = {'loan_default': 'employee_code_id_ftg'})
    df_combo_state_pin_ftg = pd.DataFrame(train_df.groupby(['State_ID','Current_pincode_ID'])['loan_default']\
                                          .apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0))\
                                            .rename(columns = {'loan_default': 'state_pin_code_ftg'})
    df_combo_state_pin_supp_ftg = pd.DataFrame(train_df.groupby(['State_ID','Current_pincode_ID','supplier_id'])['loan_default']\
                                          .apply(lambda x: x.sum()/x.count() if x.count() >=20 else 0))\
                                            .rename(columns = {'loan_default': 'state_pin_code_supp_ftg'})
        
    train_df['supplier_id_ftg'] = df_supplier_id_ftg.loc[train_df.supplier_id ,'supplier_id_ftg'].values
    train_df['branch_id_ftg'] = df_branch_id_ftg.loc[train_df.branch_id ,'branch_id_ftg'].values
    train_df['manufacturer_id_ftg'] = df_manufacturer_id_ftg.loc[train_df.manufacturer_id ,'manufacturer_id_ftg'].values
    train_df['pincode_id_ftg'] = df_pincode_id_ftg.loc[train_df.Current_pincode_ID ,'pincode_id_ftg'].values
    train_df['state_id_ftg'] = df_state_id_ftg.loc[train_df.State_ID ,'state_id_ftg'].values
    train_df['employee_code_id_ftg'] = df_employee_code_id_ftg.loc[train_df.Employee_code_ID ,'employee_code_id_ftg'].values
    train_df = pd.merge(train_df, df_combo_state_pin_ftg.reset_index(), on = ['State_ID','Current_pincode_ID'], how = 'left')
    train_df = pd.merge(train_df, df_combo_state_pin_supp_ftg.reset_index(), on = ['State_ID','Current_pincode_ID','supplier_id'], how = 'left')

    val['supplier_id_ftg'] = df_supplier_id_ftg.loc[val.supplier_id ,'supplier_id_ftg'].values
    val['branch_id_ftg'] = df_branch_id_ftg.loc[val.branch_id ,'branch_id_ftg'].values
    val['manufacturer_id_ftg'] = df_manufacturer_id_ftg.loc[val.manufacturer_id ,'manufacturer_id_ftg'].values
    val['pincode_id_ftg'] = df_pincode_id_ftg.loc[val.Current_pincode_ID ,'pincode_id_ftg'].values
    val['state_id_ftg'] = df_state_id_ftg.loc[val.State_ID ,'state_id_ftg'].values
    val['employee_code_id_ftg'] = df_employee_code_id_ftg.loc[val.Employee_code_ID ,'employee_code_id_ftg'].values
    val = pd.merge(val, df_combo_state_pin_ftg.reset_index(), on = ['State_ID','Current_pincode_ID'], how = 'left')
    val = pd.merge(val, df_combo_state_pin_supp_ftg.reset_index(), on = ['State_ID','Current_pincode_ID','supplier_id'], how = 'left')

    test_df['supplier_id_ftg'] = df_supplier_id_ftg.loc[test_df.supplier_id ,'supplier_id_ftg'].values
    test_df['branch_id_ftg'] = df_branch_id_ftg.loc[test_df.branch_id ,'branch_id_ftg'].values
    test_df['manufacturer_id_ftg'] = df_manufacturer_id_ftg.loc[test_df.manufacturer_id ,'manufacturer_id_ftg'].values
    test_df['pincode_id_ftg'] = df_pincode_id_ftg.loc[test_df.Current_pincode_ID ,'pincode_id_ftg'].values
    test_df['state_id_ftg'] = df_state_id_ftg.loc[test_df.State_ID ,'state_id_ftg'].values
    test_df['employee_code_id_ftg'] = df_employee_code_id_ftg.loc[test_df.Employee_code_ID ,'employee_code_id_ftg'].values
    test_df = pd.merge(test_df, df_combo_state_pin_ftg.reset_index(), on = ['State_ID','Current_pincode_ID'], how = 'left')
    test_df = pd.merge(test_df, df_combo_state_pin_supp_ftg.reset_index(), on = ['State_ID','Current_pincode_ID','supplier_id'], how = 'left')
    
    return train_df,val,test_df

In [21]:
train_df['branch_id'] =train_df['branch_id'].astype('int32')
train_df['supplier_id'] =train_df['supplier_id'].astype('int32')
train_df['manufacturer_id'] =train_df['manufacturer_id'].astype('int32')
train_df['Current_pincode_ID'] =train_df['Current_pincode_ID'].astype('int32')

train_df['State_ID'] =train_df['State_ID'].astype('int32')
train_df['Employee_code_ID'] =train_df['Employee_code_ID'].astype('int32')


test_df['branch_id'] =test_df['branch_id'].astype('int32')
test_df['supplier_id'] =test_df['supplier_id'].astype('int32')
test_df['manufacturer_id'] =test_df['manufacturer_id'].astype('int32')
test_df['Current_pincode_ID'] =test_df['Current_pincode_ID'].astype('int32')

test_df['State_ID'] =test_df['State_ID'].astype('int32')
test_df['Employee_code_ID'] =test_df['Employee_code_ID'].astype('int32')


In [35]:
# train_df.dtypes
test_df.isnull().sum()

UniqueID                                  0
disbursed_amount                          0
asset_cost                                0
ltv                                       0
branch_id                                 0
supplier_id                               0
manufacturer_id                           0
Current_pincode_ID                        0
Date.of.Birth                             0
Employment.Type                        3443
DisbursalDate                             0
State_ID                                  0
Employee_code_ID                          0
MobileNo_Avl_Flag                         0
Aadhar_flag                               0
PAN_flag                                  0
VoterID_flag                              0
Driving_flag                              0
Passport_flag                             0
PERFORM_CNS.SCORE                         0
PERFORM_CNS.SCORE.DESCRIPTION             0
PRI.NO.OF.ACCTS                           0
PRI.ACTIVE.ACCTS                

### Creating Data for K fold Cross Validation 
 Important point to note is that there should be no transfer of information between train and validation set 

In [22]:
skf = StratifiedKFold(n_splits= 5, shuffle= True,random_state= 5)


In [27]:
def create_bins(df,varname, nbins= 15):
    df["catg"+varname] = pd.cut(df.varname, nbins)

In [61]:
count = 1
for (trn_idx,val_idx) in skf.split(train_df,train_df['loan_default']):
    train_x,val_x = train_df.iloc[trn_idx], train_df.iloc[val_idx]
    train_y,val_y= train_df.iloc[trn_idx]['loan_default'],train_df.iloc[val_idx]['loan_default'] 
    
    
    print(train_x.shape, train_y.shape)
    train_x,val_x,test = creating_dfg_variables(train_x,val_x,test_df)
    catg_list= ['branch_id_ftg','supplier_id_ftg','manufacturer_id_ftg','pincode_id_ftg','state_id_ftg','employee_code_id_ftg']
    for col in catg_list:
        ser , bins = pd.qcut(train_x[col],15, retbins=True,duplicates= 'drop')
        print(ser,bins)
        train_x["catg_"+col] = pd.cut(train_x[col], bins = bins)
        val_x["catg_"+col] = pd.cut(val_x[col], bins = bins)
        test["catg_"+col] = pd.cut(test[col], bins = bins)
    
    train_x.to_pickle(".//CV_dataset//"+"train_"+str(count)+".csv")
    val_x.to_pickle(".//CV_dataset//"+"val_"+str(count)+".csv")
    test.to_pickle(".//CV_dataset//"+"test_"+str(count)+".csv")
    count+=1


(186522, 52) (186522,)
0         (0.185, 0.191]
1         (0.185, 0.191]
2         (0.185, 0.191]
3         (0.185, 0.191]
4         (0.185, 0.191]
5         (0.185, 0.191]
6         (0.185, 0.191]
7         (0.185, 0.191]
8         (0.185, 0.191]
9         (0.185, 0.191]
10        (0.185, 0.191]
11        (0.185, 0.191]
12        (0.185, 0.191]
13        (0.185, 0.191]
14        (0.185, 0.191]
15        (0.185, 0.191]
16        (0.185, 0.191]
17        (0.185, 0.191]
18        (0.185, 0.191]
19        (0.185, 0.191]
20        (0.185, 0.191]
21        (0.185, 0.191]
22        (0.185, 0.191]
23        (0.185, 0.191]
24        (0.185, 0.191]
25        (0.185, 0.191]
26        (0.185, 0.191]
27        (0.185, 0.191]
28        (0.185, 0.191]
29        (0.185, 0.191]
               ...      
186492    (0.236, 0.276]
186493    (0.236, 0.276]
186494    (0.236, 0.276]
186495    (0.236, 0.276]
186496    (0.236, 0.276]
186497    (0.236, 0.276]
186498    (0.236, 0.276]
186499    (0.236, 0.276]
18

In [62]:
train_x.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,state_id_ftg,employee_code_id_ftg,state_pin_code_ftg,state_pin_code_supp_ftg,catg_branch_id_ftg,catg_supplier_id_ftg,catg_manufacturer_id_ftg,catg_pincode_id_ftg,catg_state_id_ftg,catg_employee_code_id_ftg
0,537409,47145,65550,73.23,67,22807,45,1502,1985-07-31,Self employed,...,0.204947,0.404762,0.000000,0.0,"(0.191, 0.195]","(0.339, 0.741]","(0.221, 0.228]",NaN,"(0.203, 0.205]","(0.353, 0.645]"
1,417566,53278,61360,89.63,67,22807,45,1497,1985-08-24,Self employed,...,0.204947,0.404762,0.243902,0.0,"(0.191, 0.195]","(0.339, 0.741]","(0.221, 0.228]","(0.241, 0.268]","(0.203, 0.205]","(0.353, 0.645]"
2,624493,57513,66113,88.48,67,22807,45,1501,1993-12-30,Self employed,...,0.204947,0.404762,0.000000,0.0,"(0.191, 0.195]","(0.339, 0.741]","(0.221, 0.228]",NaN,"(0.203, 0.205]","(0.353, 0.645]"
3,529269,46349,61500,76.42,67,22807,45,1502,1988-01-06,Salaried,...,0.204947,0.404762,0.000000,0.0,"(0.191, 0.195]","(0.339, 0.741]","(0.221, 0.228]",NaN,"(0.203, 0.205]","(0.353, 0.645]"
4,510278,43894,61900,71.89,67,22807,45,1501,1989-04-10,Salaried,...,0.204947,0.404762,0.000000,0.0,"(0.191, 0.195]","(0.339, 0.741]","(0.221, 0.228]",NaN,"(0.203, 0.205]","(0.353, 0.645]"


In [66]:
train_x[train_x.catg_state_id_ftg.isnull()]['state_id_ftg'].value_counts()

train_x[train_x.catg_employee_code_id_ftg.isnull()]['employee_code_id_ftg'].value_counts()

train_x[train_x.catg_manufacturer_id_ftg.isnull()]['manufacturer_id_ftg'].value_counts()


train_x[train_x.catg_supplier_id_ftg.isnull()]['supplier_id_ftg'].value_counts()

In [73]:
train_x.groupby('catg_pincode_id_ftg')['loan_default'].apply(lambda x:x.sum()/x.count())

catg_pincode_id_ftg
(0.0, 0.111]      0.086646
(0.111, 0.138]    0.126931
(0.138, 0.157]    0.148100
(0.157, 0.174]    0.166238
(0.174, 0.188]    0.181213
(0.188, 0.2]      0.194133
(0.2, 0.213]      0.206368
(0.213, 0.223]    0.217807
(0.223, 0.241]    0.232682
(0.241, 0.268]    0.255311
(0.268, 0.299]    0.283729
(0.299, 0.345]    0.318903
(0.345, 0.696]    0.401084
Name: loan_default, dtype: float64

In [74]:
val_x.groupby('catg_pincode_id_ftg')['loan_default'].apply(lambda x:x.sum()/x.count())

catg_pincode_id_ftg
(0.0, 0.111]      0.160196
(0.111, 0.138]    0.163245
(0.138, 0.157]    0.160038
(0.157, 0.174]    0.184055
(0.174, 0.188]    0.194847
(0.188, 0.2]      0.189412
(0.2, 0.213]      0.213987
(0.213, 0.223]    0.208347
(0.223, 0.241]    0.221724
(0.241, 0.268]    0.231490
(0.268, 0.299]    0.246914
(0.299, 0.345]    0.294694
(0.345, 0.696]    0.334757
Name: loan_default, dtype: float64